In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [8]:
VGG = [64 , 64 , 'N', 128 , 128 , 'N', 256 , 256 , 256 , 'N', 512 , 512 , 512 , 'N', 512 , 512 , 512 , 'N']

In [9]:
class VGG_Net(nn.Module):

  def __init__(self , in_channel=3 ,num_classes=1000 ):
    super(VGG_Net, self).__init__()
    self.in_channels = in_channel
    self.conv_layers = self.create_conv_layers(VGG)
    #flatten fully connected
    self.fc = nn.Sequential(nn.Linear(512*7*7 , 4096),
                            nn.ReLU(),
                            nn.Dropout(p=0.5),
                            nn.Linear(4096 , 4096),
                            nn.ReLU(),
                            nn.Dropout(p=0.5),
                            nn.Linear(4096 , num_classes))


  def forward(self , x):
    x = self.conv_layers(x)
    x = x.reshape(x.shape[0] , -1)
    x = self.fc(x)
    return x

  def create_conv_layers(self , architecture):
    layers = []
    in_channels = self.in_channels

    for x in architecture:
      if type(x) == int:
        out_channels = x
        layers += [nn.Conv2d(in_channels=in_channels ,
                             out_channels=out_channels ,
                             kernel_size=(3,3) ,
                             stride=(1,1) ,
                             padding=(1,1)) ,
                             nn.BatchNorm2d(x) ,
                             nn.ReLU()]
        in_channels = x
      elif x == 'N':
        layers += [nn.MaxPool2d(kernel_size=(2,2) , stride=(2,2))]
    return nn.Sequential(*layers)


In [10]:
model = VGG_Net(in_channel=3 , num_classes=1000)
x = torch.randn(1,3,224,224)
print(model(x).shape)

torch.Size([1, 1000])
